In [ ]:
#export
import picos
import numpy as np
from bounce.utils import state2str, simplify_layout

In [ ]:
# default_exp sdp

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Semidefinite programming 

> Toolkit to solve the semi-definite program defined by the set of constraints and Hamiltonian.

## SdP formulation and optimization

In order to formulate and solve the SdP there are two main needed items: 
* a Hamiltonian that can be expressed in terms of `picos.Constant` through a `Hamiltonian.to_sdp()` call.
* a layout based in `np.array`, e.g., `L = [np.array([0, 1]), np.array([0, 1, 2])` determining the constraints.

In [ ]:
#export
def solve_sdp(layout, hamiltonian):
    "Solves the SDP defined by the given layout and Hamiltonian."
    layout = simplify_layout(layout)
    H = hamiltonian.to_sdp()
    problem = picos.Problem(solver = 'cvxopt')
    variables = [(site, picos.HermitianVariable('rho'+','.join(map(str, site)), (2**len(site), 2**len(site)))) for site in layout]
    problem.add_list_of_constraints([rho >> 0 for _, rho in variables])
    problem.add_list_of_constraints([picos.trace(rho) == 1 for _, rho in variables])
    
    # Energy
    objective = 0
    for support, h in H:
        supported = False
        for sites, rho in variables:
            common, _, idx = np.intersect1d(support, sites, return_indices=True)
            if len(common) == len(support): 
                rdm = rho.partial_trace(complementary_system(idx, len(sites)))
                objective += (rdm | h) # Tr(rdm·H')
                supported = True
                break
        if not supported:
            eigenvalues, _ = np.linalg.eigh(picos2np(h))
            objective += min(eigenvalues)
                
    problem.set_objective('min', objective)
    
    # Compatibility
    compatibility_constraints = []
    for k1, (sites1, rho1) in enumerate(variables):
        for k2 in range(k1+1, len(variables)):
            sites2, rho2 = variables[k2]
            common, idx1, idx2 = np.intersect1d(sites1, sites2, return_indices=True)
            if len(common) > 0:
                partial_trace1 = rho1.partial_trace(complementary_system(idx1, len(sites1)))
                partial_trace2 = rho2.partial_trace(complementary_system(idx2, len(sites2)))
                constraint = partial_trace1 - partial_trace2 == 0
                compatibility_constraints.append(constraint)
    
    problem.add_list_of_constraints(compatibility_constraints)
    
    try:    
        problem.solve()
        result = np.real(objective.value)
    except: 
        print(problem)
        result = 0.
    return result
    
def complementary_system(subsystem, size):
    "Obtains the complementary system of subsystem."
    return list(map(int, np.setdiff1d(np.arange(size), subsystem)))

def picos2np(variable):
    "Converts picos variable (even sparse) to numpy matrix."
    return picos.expressions.data.cvx2np(variable.value)

In [ ]:
#hide
from bounce.hamiltonian import XXHamiltonian
from bounce.utils import fill_layout

The SdP is built imposing the compatibility contraints from the layout. There, we simply provide a list of `np.array` containing the support for each reduced density matrix (RDM) that we consider. In the following example, we solve the SdP given the example Hamiltonian and imposing compatibility over a single pair of sites. We will consider the RDMs $\rho_0, \rho_1, \rho_2, \rho_3, \rho_4, \rho_{50}$ in a ring with 6 sites, which is the same as the set of constraints $C=\{\{0\},\{1\},\{2\},\{3\},\{4\},\{5,0\}\}$. The function `fill_layout` allows us to skip the 1-body terms when providing the set of constraints, as it will fill any site that is not contained within the provided constraints with its corresponding 1-body term.   

In [ ]:
N = 6
B, J = [1]*N, [i%3 for i in range(N)]
H = XXHamiltonian(N, B, J)

With the Hamiltonian, define the set of constraints.

In [ ]:
simple_layout = fill_layout([np.array([0, N-1])], N)
simple_layout

[array([0, 5]), array([1]), array([2]), array([3]), array([4])]

And now we can solve the associated SdP to the Hamiltonian with the given set of constraints.

In [ ]:
solve_sdp(simple_layout, H)

-15.999999999012134

Tightening the constraints we can obtain a better energy bound. For instance, adding a 3-body constraint to the previous set.

In [ ]:
stronger_layout = fill_layout([np.array([1, 2, 3]), np.array([0, N-1])], N)
stronger_layout

[array([1, 2, 3]), array([0, 5]), array([4])]

In [ ]:
solve_sdp(stronger_layout, H)

-12.4721359498209

## Cost estimation

In order to have an estimation of the computational cost required to solve a given SdP, we estimate the amount of free variables in the optimization problem. 

In [ ]:
#export
def ojimetro(L):
    "Estimates the amount of free parameters in the SDP associated to the layout."
    # The old blocks are len(L)
    L = simplify_layout(L)
    all_variables = np.sum([2**(2*len(sites)) for sites in L])
    intersections = []
    for k, sites1 in enumerate(L[:-1]):
        for sites2 in L[k+1:]:
            intersections.append(np.intersect1d(sites1, sites2))
    intersections = simplify_layout(intersections)
    dep_variables = np.sum([2**(2*len(sites)) for sites in intersections])
    return all_variables-dep_variables   

Following with our previous example, the first set of constraints was rather loose, providing a low energy bound. However, the second set of constraints provided a tighter bound, although the SdP was harder to solve. 

In [ ]:
ojimetro(simple_layout)

31

In [ ]:
ojimetro(stronger_layout)

83

With the first set of constraints, the resulting SdP had 31 free variables to optimize, while the second SdP had to deal with 83. Tighter energy bounds usually come at the cost of higher computational costs. 

## Other methods
We implement two other methods to obtain lower bounds of many-body Hamiltonians. We rewrite the Anderson bound [[1]](https://journals.aps.org/pr/abstract/10.1103/PhysRev.83.1260) and the method introduced by Uskov and Lichovskiy in [[2]](https://iopscience.iop.org/article/10.1088/1742-6596/1163/1/012057) in SdP form to compare with the proposed methodology above. The SdP formulation ensures that the obtained result is the actual global minimum.

The main limitation of these methods is not taking into account compatibility constraints between the reduced density matrices spanning the system. Furthermore, they rely on symmetries (such as translational invariance) and can, thus, not be used to solve inhomogeneous systems. Conversely, the approach we introduced above can be nautrally complemented by introducing additional constratints stemming from any previously known symmetry from the system. 

In [ ]:
#export
def anderson_bound(hamiltonian, cluster_size=3):
    "Provides the corresponding Anderson bound to the Hamiltonian given a cluster size."
    if hamiltonian.N % (cluster_size-1) != 0:
        raise ValueError(f"The number of qubits {hamiltonian.N} must be divisible by cluster_size - 1")
    else: 
        n_clusters = hamiltonian.N // (cluster_size-1) # Assumes 1D nearest-neighbor interaction
    H = hamiltonian.to_sdp()
    problem = picos.Problem(solver = 'cvxopt')
    
    sites = np.arange(cluster_size)
    rho = picos.HermitianVariable('rho'+','.join(map(str, sites)), (2**cluster_size, 2**cluster_size))
    problem.add_constraint(rho >> 0)
    problem.add_constraint(picos.trace(rho) == 1)
    
    # Energy
    objective = 0
    for support, h in H:
        common, _, idx = np.intersect1d(support, sites, return_indices=True)
        if len(common) == len(support): 
            rdm = rho.partial_trace(complementary_system(idx, len(sites)))
            objective += (rdm | h) # Tr(rdm·H')
    
    problem.set_objective('min', objective)
    
    try:    
        problem.solve()
        result = np.real(objective.value)
    except: 
        print(problem)
        result = 0.
    return n_clusters*result

def uskov_lichkovskiy_bound(hamiltonian, cluster_size=3):
    "Provides the corresponding Uskov and Lichkovskiy bound to the Hamiltonian given a cluster size."
    if hamiltonian.N % (cluster_size-1) != 0:
        raise ValueError(f"The number of qubits {hamiltonian.N} must be divisible by cluster_size - 1")
    else: 
        n_clusters = hamiltonian.N // (cluster_size-1) # Assumes 1D nearest-neighbor interaction
    H = hamiltonian.to_sdp()
    problem = picos.Problem(solver = 'cvxopt')
    
    sites = np.arange(cluster_size)
    rho = picos.HermitianVariable('rho'+','.join(map(str, sites)), (2**cluster_size, 2**cluster_size))
    problem.add_constraint(rho >> 0)
    problem.add_constraint(picos.trace(rho) == 1)

    # Symmetry constraints: TI and specular symmetry in the cluster
    x = picos.Constant('x', [[0, 1], [1, 0]])
    y = picos.Constant('y', [[0, -1j], [1j, 0]])
    z = picos.Constant('z', [[1, 0], [0, -1]])
    Id = picos.Constant('Id', [[1, 0], [0, 1]])
    
    def tensor_prod(idx, s):
        "Tensor product of `s` acting on indexes `idx`. Fills rest with Id."
        matrices = [Id if k not in idx else s for k in range(cluster_size)]
        prod = matrices[0]
        for k in range(1, cluster_size): prod = prod @ matrices[k]
        return prod
    
    def sigma_term(idx): return tensor_prod(idx, x) + tensor_prod(idx, y) + tensor_prod(idx, z)
            
    idx = [sites[i:i+2] for i in range(len(sites)-1)]
    for i in range(len(idx)//2):
        left, right = sigma_term(idx[i]), sigma_term(idx[-(i+1)])
        problem.add_constraint((left | rho) == (right | rho)) # Tr(sigma_01*rho) = Tr(sigma_{n-1,n}*rho)
        adj = sigma_term(idx[i+1])
        problem.add_constraint((left | rho) == (adj | rho)) # Tr(sigma_01*rho) = Tr(sigma_12*rho)
    
    # Energy
    objective = 0
    for support, h in H:
        common, _, idx = np.intersect1d(support, sites, return_indices=True)
        if len(common) == len(support): 
            rdm = rho.partial_trace(complementary_system(idx, len(sites)))
            objective += (rdm | h) # Tr(rdm·H')
    
    problem.set_objective('min', objective)
    
    try:    
        problem.solve()
        result = np.real(objective.value)
    except: 
        print(problem)
        result = 0.
    return n_clusters*result

In [ ]:
#hide
from bounce.hamiltonian import XYZHamiltonian

The results from [[2]](https://iopscience.iop.org/article/10.1088/1742-6596/1163/1/012057) can be reproduced by properly adjusting the cluster size and using the following Hamiltonian.

In [ ]:
N = 8
linear, quadratic = [0]*N, [1]*N
hamiltonian = XYZHamiltonian(N, linear, quadratic)

In [ ]:
cluster_size = 5
bound_anderson = anderson_bound(hamiltonian, cluster_size=cluster_size)
bound_uskov = uskov_lichkovskiy_bound(hamiltonian, cluster_size=cluster_size)

layout = [np.sort(np.arange(i, i+cluster_size)%N) for i in np.arange(0, N, cluster_size-3)]
bound_ours = solve_sdp(layout, hamiltonian)

bound_anderson/N, bound_uskov/N, bound_ours/N

(-1.9278862525095595, -1.8685170845449277, -1.868517086761974)

## References
[1] P. W. Anderson. [Limits on the Energy of the Antiferromagnetic Ground State](https://journals.aps.org/pr/abstract/10.1103/PhysRev.83.1260). *Physical Review* **83**, 1260 (1951)

[2] F. Uskov and O. Lychkovskiy. [A variational lower bound on the ground state of a many-body system and the squaring parametrization of density matrices](https://iopscience.iop.org/article/10.1088/1742-6596/1163/1/012057). *Journal of Physics: Conference Series* **1163** 012057 (2019)

## Export-

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_environment.ipynb.
Converted 01_agents.ipynb.
Converted 02_budget_profiles.ipynb.
Converted 03_hamiltonian.ipynb.
Converted 04_training.ipynb.
Converted 05_utils.ipynb.
Converted 06_sdp.ipynb.
Converted index.ipynb.
